In [7]:
import _pickle
def preprocess(words_file = "tools/word_data.pkl", authors_file="tools/email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features

        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions

        4 objects are returned:
            -- training/testing features
            -- training/testing labels

    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, "rb")
    authors = pickle.load(authors_file_handler)
    authors_file_handler.close()

    words_file_handler = open(words_file, "rb")
    word_data = pickle.load(words_file_handler)
    words_file_handler.close()

    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size= 0.14, random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    #print(vectorizer.vocabulary_)
    #print(vectorizer.idf_)
    
    features_test_transformed  = vectorizer.transform(features_test)



    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print ("no. of Chris training emails:", sum(labels_train))
    print ("no. of Sara training emails:", len(labels_train)-sum(labels_train))
    
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test


### Q : How TfidfVectorizer works ? And how tf-idf is calculated?

In order to explain this we first need to know bags of word representation and understand what numerical  feature vector is . 
Bags of wordsrepresentation  is perforemed in 3 steps

document = ['hello beautiful people hello'] 

**1) Tokenizing** : Make tokens and id them.

['hello' , 'beautiful' , 'people', 'hello'] 

hello:     id-10 <br>
beautiful: id-20 <br>
people:    id-24 <br>

**result :** [id-10, id-20 ,id-24 ,id-10]

**2) counting : ** <br>
id-10: 2 times  <br>
id-20: 1 time<br>
id-24:1 time <br>
**result: **[2, 1, 1]

3)** Normalize and weighting** of less important tokens that occur frequently and dont offer information about a document.<br>
This is where tf-ifd  plays its part, if you want to know more about this  , read "4.2.3. Text feature extraction" from this link 

http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction


Numerical feature vector is vector we get after counting.we use this vector to compute tfidf.

**optional:**

https://www.quora.com/How-does-TfidfVectorizer-work-in-laymans-terms
***

### Q : what do fit ,transform and fit_transform do ?

https://datascience.stackexchange.com/questions/12321/difference-between-fit-and-fit-transform-in-scikit-learn-models

In [8]:
import pickle
from time import time

import numpy

from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif


### features_train and features_test are the features for the training
### and testing datasets, respectively
### labels_train and labels_test are the corresponding item labels
features_train, features_test, labels_train, labels_test = preprocess()
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

t0 = time()
clf.fit(features_train,labels_train)
print ("training time:", round(time()-t0, 3), "s")

t0 = time()
y_pred = clf.predict(features_test)
print ("training time:", round(time()-t0, 3), "s")


from sklearn.metrics import accuracy_score
print (accuracy_score(labels_test, y_pred))

no. of Chris training emails: 7571
no. of Sara training emails: 7546
training time: 1.488 s
training time: 0.311 s
0.9744006501422187


I notices when 

test set = 10 % , accuracy= 0.9732650739476678 <br>
test set = 14 % , aacuracy= 0.9744006501422187

So i canged it to 14 %. 